## Download other market data 

For comparison purposes, download additional market data from noncryptocurrency sources.

This code was modified from a CS109b Project by Asher Diamant and Paul Washburn that was supervised by David Wihl

It is intended to run once to download these additional data sources:

* S&P 500
* NASDAQ
* DOW
* RUSSELL 2000
* Monetary Base Total (BOGMBASEW)
* Japan US Foreign Exchange Rate (DEXJPUS)
* VIX Index
* etc.

Details of these different metrics can be found at `https://fred.stlouisfed.org/series/METRICNAME` e.g.

[https://fred.stlouisfed.org/series/BAMLHYH0A0HYM2TRIV](https://fred.stlouisfed.org/series/BAMLHYH0A0HYM2TRIV)

In [1]:
import pandas as pd
import numpy as np
import quandl
import time
import os
import pandas_datareader.data as web


In [2]:
# get stock indices
sp_ix = web.get_data_fred('SP500')  # S&P 500
nq_ix = web.get_data_fred('NASDAQCOM')  # NASDAQ
dow_ix = web.get_data_fred('DJIA') # DOW
russ_ix = web.get_data_fred('RU2000PR') # RUSSELL 2000

In [3]:
# compile to one dataframe
def merge_on_index(df_list):
    new_df = df_list[0]
    for df in df_list[1:]:
        new_df = new_df.join(df)
    return new_df

# merge in equities
df_list = [sp_ix, nq_ix, dow_ix, russ_ix]
equities_df = merge_on_index(df_list)
combined_df = equities_df

# Get currency and VIX data
# monetary base USD
usd_base = web.get_data_fred('BOGMBASEW')

# exchange rates
us_jp = web.get_data_fred('DEXJPUS')
us_eu = web.get_data_fred('DEXUSEU')
us_eu = np.divide(1, us_eu) # put into terms of other currency
us_ch = web.get_data_fred('DEXCHUS')
us_mx = web.get_data_fred('DEXMXUS')
us_au = web.get_data_fred('DEXUSAL')
us_au = np.divide(1, us_au) # put into terms of other currency

# VIX index
vix = web.get_data_fred('VIXCLS').dropna().squeeze()

# merge in currencies
df_list = [us_jp, us_eu, us_ch, us_mx, us_au, vix]
currency_df = merge_on_index(df_list)
combined_df = combined_df.join(currency_df)

# LIBOR
libor_ovnt_usd = web.get_data_fred('USDONTD156N')
libor_01mo_usd = web.get_data_fred('USD1MTD156N')
libor_03mo_usd = web.get_data_fred('USD3MTD156N')
libor_12mo_usd = web.get_data_fred('USD12MD156N')

# high yield
high_yield_ix = web.get_data_fred('BAMLHYH0A0HYM2TRIV')
corp_aaa_ix = web.get_data_fred('BAMLCC0A1AAATRIV')

df_list = [libor_ovnt_usd, libor_01mo_usd, libor_03mo_usd, libor_12mo_usd,
          high_yield_ix, corp_aaa_ix]
interest_rate_df = merge_on_index(df_list)
combined_df = combined_df.join(interest_rate_df)


# commodity prices
daily_gold_usd = web.get_data_fred('GOLDAMGBD228NLBM')
daily_wticrude_usd = web.get_data_fred('DCOILWTICO')

df_list = [daily_gold_usd, daily_wticrude_usd]
commodity_df = merge_on_index(df_list)
combined_df = combined_df.join(commodity_df)

In [4]:
combined_df.index = pd.DatetimeIndex(combined_df.index)
# delete all prior to 2015-08-07 or after 2018-02-20 to match with Cryptocurrency data
combined_df = combined_df[(combined_df.index >= '2015-08-07') &  (combined_df.index <= '2018-02-20')]


In [5]:
combined_df.head()

,SP500,NASDAQCOM,DJIA,RU2000PR,DEXJPUS,DEXUSEU,DEXCHUS,DEXMXUS,DEXUSAL,VIXCLS,USDONTD156N,USD1MTD156N,USD3MTD156N,USD12MD156N,BAMLHYH0A0HYM2TRIV,BAMLCC0A1AAATRIV,GOLDAMGBD228NLBM,DCOILWTICO
DATE,,,,,,,,,,,,,,,,,,
2015-08-07,2077.57,5043.54,17373.38,2999.45,124.27,0.912575,6.2087,16.1780,1.352631,13.39,0.1260,0.19125,0.31160,0.83585,1059.15,569.40,1091.35,43.87
2015-08-10,2104.18,5101.80,17615.17,3038.65,124.55,0.909587,6.2094,16.1535,1.350621,12.23,0.1265,0.19255,0.31420,0.84670,1058.73,565.85,1094.80,44.94
2015-08-11,2084.07,5036.79,17402.84,3009.98,124.90,0.905633,6.3232,16.3335,1.372119,13.71,0.1275,0.19345,0.31435,0.83870,1054.53,570.20,1113.25,43.11
2015-08-12,2086.05,5044.39,17402.51,3004.62,123.87,0.893017,6.3845,16.2960,1.355932,13.61,0.1280,0.19400,0.30930,0.82215,1050.85,569.30,1116.80,43.22
2015-08-13,2083.39,5033.56,17408.25,2994.08,124.32,0.897344,6.3982,16.3735,1.358511,13.49,0.1285,0.19760,0.32050,0.83490,1053.10,566.78,1117.35,42.27


In [6]:
combined_df.tail()

,SP500,NASDAQCOM,DJIA,RU2000PR,DEXJPUS,DEXUSEU,DEXCHUS,DEXMXUS,DEXUSAL,VIXCLS,USDONTD156N,USD1MTD156N,USD3MTD156N,USD12MD156N,BAMLHYH0A0HYM2TRIV,BAMLCC0A1AAATRIV,GOLDAMGBD228NLBM,DCOILWTICO
DATE,,,,,,,,,,,,,,,,,,
2018-02-14,2698.63,7143.62,24893.49,3782.79,107.00,0.806712,6.3438,18.6035,1.270164,19.26,1.44375,1.58813,1.85000,2.33844,1246.97,609.96,1330.75,60.70
2018-02-15,2731.20,7256.43,25200.37,3820.32,106.36,0.801154,6.3438,18.5195,1.263903,19.13,1.44375,1.59000,1.87250,2.38125,1253.04,612.67,1353.70,61.48
2018-02-16,2732.22,7239.47,25219.38,3836.11,106.10,0.803729,6.3438,18.4780,1.261830,19.46,1.44500,1.59375,1.88494,2.39063,1256.91,613.54,1358.60,61.89
2018-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.59563,1.89213,2.39813,1257.56,613.71,1347.40,NaN
2018-02-20,2716.26,7234.31,24964.75,3802.42,107.23,0.809848,6.3351,18.6390,1.265342,20.60,1.44500,1.59563,1.90394,2.40844,1258.05,612.53,1337.40,61.91


In [7]:
# Fill N/A values (holidays, weekends, time before coin started trading)
combined_df = combined_df.fillna(method='ffill').fillna(method='bfill')
fname = 'data/noncrypto.csv'
print('Writing file to {}'.format(fname))
combined_df.to_csv(fname)


Writing file to data/noncrypto.csv
